OS 및 라이브러리 버전
* Python 3.10.12
* pandas 1.5.3
* numpy 1.23.5
* catboost 1.2.2
* sklearn 1.2.2

In [ ]:
!pip install catboost

# drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

# import


In [ ]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostClassifier
import sklearn
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

###OS 및 라이브러리 버전

In [1]:
!cat /etc/issue
!python --version
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('catboost', catboost.__version__)
print('sklearn', sklearn.__version__)

Ubuntu 22.04.3 LTS \n \l

Python 3.10.12


NameError: ignored

# 데이터 불러오기


In [ ]:
# 데이터 불러오기
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

# 데이터 전처리

## test에 존재하는 column 선택

In [ ]:
categorical_features = ['COMPONENT_ARBITRARY', 'YEAR', 'YEAR_COMPONENT']
# Inference(실제 진단 환경)에 사용하는 컬럼
test_stage_features = ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR' , 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

## 결측치 처리

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

## feature 생성

In [ ]:
train_y = train['Y_LABEL']
train = train[test_stage_features]

metal = ['AG', 'CO', 'CR', 'CU', 'FE', 'MN', 'MO', 'NI', 'TI', 'V', 'ZN']

train['YEAR_COMPONENT'] = train[['YEAR','COMPONENT_ARBITRARY']].apply(lambda x: '-'.join(x.astype(str)),axis=1)
test['YEAR_COMPONENT'] = test[['YEAR','COMPONENT_ARBITRARY']].apply(lambda x: '-'.join(x.astype(str)),axis=1)

train_metal = train[metal]
test_metal = test[metal]

train['metal_mean'] = train_metal.mean(axis = 1)
train['metal_median'] = train_metal.median(axis = 1)
train['metal_max'] = train_metal.max(axis = 1)
train['metal_min'] = train_metal.min(axis = 1)

test['metal_mean'] = test_metal.mean(axis = 1)
test['metal_median'] = test_metal.median(axis = 1)
test['metal_max'] = test_metal.max(axis = 1)
test['metal_min'] = test_metal.min(axis = 1)

In [ ]:
train

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,...,PQINDEX,TI,V,V40,ZN,YEAR_COMPONENT,metal_mean,metal_median,metal_max,metal_min
0,COMPONENT3,-0.996688,2011,-1.000000,0,0,13,78,888,0.0,...,8504,5,0,154.0,75,2011-COMPONENT3,98.363636,6.0,888,0
1,COMPONENT2,-0.997615,2021,-0.962963,0,0,0,31,2,0.0,...,19,0,0,44.0,652,2021-COMPONENT2,62.272727,0.0,652,0
2,COMPONENT2,-0.990356,2015,-1.000000,0,0,1,2,4,0.0,...,17,0,0,72.6,412,2015-COMPONENT2,38.090909,0.0,412,0
3,COMPONENT3,-0.956456,2010,-1.000000,0,0,0,1,37,0.0,...,44,0,0,133.3,7,2010-COMPONENT3,4.181818,0.0,37,0
4,COMPONENT3,-0.979867,2015,-1.000000,0,0,0,0,71,0.0,...,217,0,0,133.1,128,2015-COMPONENT3,18.090909,0.0,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,COMPONENT3,-0.995802,2014,-1.000000,0,0,0,3,23,0.0,...,35,0,0,135.4,16,2014-COMPONENT3,3.818182,0.0,23,0
14091,COMPONENT1,-0.987841,2013,-1.000000,0,0,0,2,12,0.0,...,9,0,0,117.5,1408,2013-COMPONENT1,149.636364,0.0,1408,0
14092,COMPONENT3,-0.994629,2008,-0.925926,0,0,4,7,415,0.0,...,645,0,0,54.0,1301,2008-COMPONENT3,158.636364,4.0,1301,0
14093,COMPONENT2,-0.989790,2009,-0.925926,0,0,0,170,19,0.0,...,11,0,0,44.3,652,2009-COMPONENT2,76.545455,0.0,652,0


In [ ]:
test

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,...,PQINDEX,TI,V,V40,ZN,YEAR_COMPONENT,metal_mean,metal_median,metal_max,metal_min
0,TEST_0000,COMPONENT1,-0.995011,2014,-1.000000,0,0,3,43,45,...,15,0,0,85.5,1007,2014-COMPONENT1,100.000000,0.0,1007,0
1,TEST_0001,COMPONENT3,-0.984052,2015,-1.000000,0,0,0,0,165,...,62,0,0,155.5,35,2015-COMPONENT3,18.363636,0.0,165,0
2,TEST_0002,COMPONENT3,-0.977618,2021,-0.903280,0,0,1,12,101,...,158,0,0,46.8,1480,2021-COMPONENT3,146.454545,1.0,1480,0
3,TEST_0003,COMPONENT1,-0.988407,2010,-1.000000,0,0,6,7,66,...,14,0,0,109.9,1023,2010-COMPONENT1,115.727273,3.0,1023,0
4,TEST_0004,COMPONENT2,-0.985626,2017,-1.000000,0,0,0,5,3,...,8,0,0,46.8,377,2017-COMPONENT2,35.000000,0.0,377,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,TEST_6036,COMPONENT1,-0.983268,2017,-1.000000,0,0,0,1,10,...,10,0,0,100.5,1180,2017-COMPONENT1,108.818182,0.0,1180,0
6037,TEST_6037,COMPONENT2,-0.985067,2012,-1.000000,0,0,0,6,11,...,14,0,0,62.5,58,2012-COMPONENT2,6.818182,0.0,58,0
6038,TEST_6038,COMPONENT3,-0.997165,2015,-0.911958,0,0,0,2,70,...,80,0,0,146.7,10,2015-COMPONENT3,7.454545,0.0,70,0
6039,TEST_6039,COMPONENT1,-0.995263,2014,-1.000000,0,0,0,0,5,...,18,0,0,99.0,909,2014-COMPONENT1,83.090909,0.0,909,0


## label encoding

In [ ]:
test = test.drop(['ID'], axis = 1)

In [ ]:
le = LabelEncoder()
for col in categorical_features:
    train[col] = le.fit_transform(train[col])
    if col in test.columns:
        test[col] = le.transform(test[col])

In [ ]:
train

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,...,V,V40,ZN,YEAR_COMPONENT,metal_mean,metal_var,metal_std,metal_median,metal_max,metal_min
0,2,-0.996688,4,-1.000000,0,0,13,78,888,0.0,...,0,154.0,75,18,98.363636,69431.054545,263.497732,6.0,888,0
1,1,-0.997615,14,-0.962963,0,0,0,31,2,0.0,...,0,44.0,652,57,62.272727,38341.218182,195.809137,0.0,652,0
2,1,-0.990356,8,-1.000000,0,0,1,2,4,0.0,...,0,72.6,412,33,38.090909,15380.490909,124.018107,0.0,412,0
3,2,-0.956456,3,-1.000000,0,0,0,1,37,0.0,...,0,133.3,7,14,4.181818,122.763636,11.079875,0.0,37,0
4,2,-0.979867,8,-1.000000,0,0,0,0,71,0.0,...,0,133.1,128,34,18.090909,1782.490909,42.219556,0.0,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,2,-0.995802,7,-1.000000,0,0,0,3,23,0.0,...,0,135.4,16,30,3.818182,63.363636,7.960128,0.0,23,0
14091,0,-0.987841,6,-1.000000,0,0,0,2,12,0.0,...,0,117.5,1408,24,149.636364,178648.654545,422.668492,0.0,1408,0
14092,2,-0.994629,1,-0.925926,0,0,4,7,415,0.0,...,0,54.0,1301,6,158.636364,158822.054545,398.524848,4.0,1301,0
14093,1,-0.989790,2,-0.925926,0,0,0,170,19,0.0,...,0,44.3,652,9,76.545455,38991.472727,197.462586,0.0,652,0


#feature selection

In [ ]:
stf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

result = []

train_x = train.copy()
train_y = train_y.copy()

n = 0

for train_index, valid_index in stf.split(train_x, train_y):
    train_X = np.array(train_x)
    train_Y = np.array(train_y)

    X_train, X_valid = train_X[train_index], train_X[valid_index]
    y_train, y_valid = train_Y[train_index], train_Y[valid_index]

    model = CatBoostClassifier(iterations = 1000, random_seed=42,
                               verbose=0)

    summary = model.select_features(
        X = X_train,
        y = y_train,
        features_for_select = '0-22',
        eval_set=(X_valid, y_valid),
        num_features_to_select=10,
        steps=13,
        train_final_model=False,
        logging_level='Silent',
        plot=False
    )

    result.append(summary['selected_features'])
    print(n)
    n+=1
print("done")

0
0
0
0
0
done


In [ ]:
print(result)
feature_se = []
for i in range(5):
    for j in result[i]:
        feature_se.append(j)
feature_se = set(feature_se)
feature_select = [train.columns[i] for i in feature_se]
print(len(feature_select))
print(feature_select)

[[0, 6, 8, 9, 11, 13, 14, 16, 18, 19], [0, 6, 7, 8, 9, 13, 14, 16, 19, 21], [0, 6, 7, 8, 9, 10, 13, 14, 16, 19], [0, 6, 7, 8, 9, 10, 13, 14, 16, 17], [0, 6, 7, 8, 9, 10, 13, 14, 16, 21]]
14
['COMPONENT_ARBITRARY', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'PQINDEX', 'TI', 'V40', 'ZN', 'YEAR_COMPONENT', 'metal_mean', 'metal_max']


In [ ]:
#catboost의 특성을 고려하여 YEAR 범주형 변수 추가
feature_select_cat = ['COMPONENT_ARBITRARY', 'YEAR', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'PQINDEX', 'TI', 'V40', 'ZN', 'YEAR_COMPONENT', 'metal_mean', 'metal_max']
len(feature_select_cat)

15

# model 학습

In [ ]:
catparam = {
    'scale_pos_weight' : 2,
    'random_state' : 42,
    'verbose': 0
}

In [ ]:
clf = CatBoostClassifier(**catparam)

train_x = train[feature_select_cat]
test_x = test[feature_select_cat]

In [ ]:
clf.fit(train_x, train_y)

# 모델 저장

In [ ]:
import pickle
with open('saved_catboost.pickle','wb') as fw:
    pickle.dump(clf, fw)

In [ ]:
with open('saved_catboost.pickle','rb') as f:
    model = pickle.load(f)

# 제출

In [ ]:
y_pred = clf.predict(test_x)

submit = pd.read_csv('./sample_submission.csv')
submit['Y_LABEL'] = y_pred
submit.to_csv('./digda_catboost.csv', index=False)